In [1]:
#El objetivo principal de este codigo es generar el cascaron intermedio de FEM 
#a partir de dos mallas superficiales interior y exterior de archivo off

# Importacion de librerias
import pygamer
print("PyGAMer version:", pygamer.__version__())
import numpy as np
import threevis
import trimesh

PyGAMer version: 2.0.5


In [2]:
# Llama a las mallas de archivos off para generar el cascaron
mesh1 = pygamer.readOFF("Sphere5R3.off") #Malla interior
mesh2 = pygamer.readOFF("Sphere6R3.off") #Malla exterior
# Corrige el vector normal de las mallas
mesh1.correctNormals()
mesh2.correctNormals()

In [3]:
#Seleccion un numero como etiqueta para referenciar las dos mallas para su visualizacion
for faceID in mesh1.faceIDs:
    faceID.data().marker = 23
for faceID in mesh2.faceIDs:
    faceID.data().marker = 50

#Si se desea una malla volumetrica afuera de la superficie de la figura, colocar True
#Si se desea una malla volumetrica dentro de la superficie de la figura, colocar False
# Informacion de la malla 1
gInfo = mesh1.getRoot()
gInfo.ishole = True 
# Informacion de la malla 2
gInfo = mesh2.getRoot()
gInfo.ishole = False

In [4]:
# Visualizacion de la malla interior y exterior
protverts, _, protfaces = mesh1.to_ndarray()
boxverts, boxedges, _ = mesh2.to_ndarray()
ctx = threevis.Context(width=640, height=480)
colors = np.zeros((len(protfaces), 3))
colors[:,0] = np.ones(len(protfaces))
ctx.draw_faces(protverts, protfaces, colors=threevis.FaceAttribute(colors))
ctx.draw_edges(boxverts, boxedges)
ctx.display()

Renderer(background='#dddddd', camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(…

In [5]:
# Construye una lista de mallas para la funcion TetGen
meshes = [mesh1,mesh2]
#Tetrahedriza la lista de mallas para generar el casacaron con TetGen.
tetmesh = pygamer.makeTetMesh(meshes, "-pq1.1aYAO2/3")
#https://custem.readthedocs.io/en/latest/_modules/custEM/meshgen/meshgen_tools.html #(Informacion de respaldo)

Number of vertices: 5656
Number of Faces: 11304
Number of Regions: 1
Number of Holes: 1
Region point: Tensor({0}:4.62641; {1}:0.169336; {2}:0.0978065)
Region point: Tensor({0}:5.62372; {1}:0.173668; {2}:0.100343)


In [6]:
#Informacion de las mallas y el cascaron
print(F"The mesh1 currently has {mesh1.nVertices} vertices, \
{mesh1.nEdges} edges, and {mesh1.nFaces} faces.")
print(F"The mesh2 currently has {mesh2.nVertices} vertices, \
{mesh2.nEdges} edges, and {mesh2.nFaces} faces.")
print(F"The tetmesh currently has {tetmesh.nVertices} vertices, \
{tetmesh.nEdges} edges, and {tetmesh.nFaces} faces.")

The mesh1 currently has 2334 vertices, 6996 edges, and 4664 faces.
The mesh2 currently has 3322 vertices, 9960 edges, and 6640 faces.
The tetmesh currently has 7614 vertices, 43564 edges, and 66252 faces.


In [7]:
#Elige el nombre y genera el archivo xml de la malla volumetrica(dentro o afuera) de la proteina
#pygamer.writeDolfin('outputTetMeshSphere56R3.xml', tetmesh)

In [8]:
#Se realiza un corte en el cascaron antes de hacer la visualizacion
toRemove = []
for vertexID in tetmesh.vertexIDs:
    if vertexID.data()[1] > 0:
        toRemove.append(vertexID)
for v in toRemove:
    tetmesh.removeVertex(v)
    
surfmesh = tetmesh.extractSurface()
surfmesh.correctNormals()
v, e, f = surfmesh.to_ndarray()

In [9]:
# Visualizacion del cascaron
ctx = threevis.Context(width=640, height=480)

rgb = np.ones((len(f), 3))

for i, face in enumerate(surfmesh.faceIDs):
    if face.data().marker == 23:
        rgb[i,0] = 1
        rgb[i,1] = 0
        rgb[i,2] = 0
    elif face.data().marker == 50:
        rgb[i,0] = 0
        rgb[i,1] = 0
        rgb[i,2] = 1

colors = threevis.FaceAttribute(rgb)

ctx.draw_faces(v, f, colors=colors)
ctx.draw_edges(v, e)
ctx.display()

Renderer(background='#dddddd', camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(…